In [1]:
from __future__ import print_function, division

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
import datetime

In [4]:
# Source: http://web.mta.info/developers/turnstile.html
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)
        
week_nums = [190629, 190622, 190615, 190608]
turnstiles_df = get_data(week_nums)

In [5]:
turnstiles_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,00:00:00,REGULAR,7107725,2407457
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,04:00:00,REGULAR,7107738,2407465
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,08:00:00,REGULAR,7107761,2407491
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,12:00:00,REGULAR,7107858,2407541
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,16:00:00,REGULAR,7108075,2407581


In [6]:
turnstiles_df.columns = [column.strip() for column in turnstiles_df.columns]
turnstiles_df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [7]:
turnstiles_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,00:00:00,REGULAR,7107725,2407457
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,04:00:00,REGULAR,7107738,2407465
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,08:00:00,REGULAR,7107761,2407491
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,12:00:00,REGULAR,7107858,2407541
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,16:00:00,REGULAR,7108075,2407581


In [8]:
#4 weeks of data
turnstiles_df.DATE.value_counts().sort_index()

06/01/2019    29066
06/02/2019    28968
06/03/2019    30022
06/04/2019    29513
06/05/2019    29484
06/06/2019    29213
06/07/2019    28745
06/08/2019    29030
06/09/2019    29133
06/10/2019    29351
06/11/2019    29113
06/12/2019    30023
06/13/2019    29276
06/14/2019    29008
06/15/2019    29605
06/16/2019    29322
06/17/2019    29421
06/18/2019    29599
06/19/2019    29614
06/20/2019    29556
06/21/2019    30360
06/22/2019    29217
06/23/2019    29170
06/24/2019    29479
06/25/2019    29437
06/26/2019    29417
06/27/2019    29464
06/28/2019    29779
Name: DATE, dtype: int64

In [10]:
from datetime import datetime as dt

In [11]:
# Take the date and time fields into a single datetime column
turnstiles_df["DATE_TIME"] = pd.to_datetime(turnstiles_df.DATE + " " + turnstiles_df.TIME, 
                                            format="%m/%d/%Y %H:%M:%S")

In [14]:
turnstiles_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,00:00:00,REGULAR,7107725,2407457,2019-06-22 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,04:00:00,REGULAR,7107738,2407465,2019-06-22 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,08:00:00,REGULAR,7107761,2407491,2019-06-22 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,12:00:00,REGULAR,7107858,2407541,2019-06-22 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,16:00:00,REGULAR,7108075,2407581,2019-06-22 16:00:00


In [17]:
# Sanity Check to verify that "C/A", "UNIT", "SCP", "STATION", "DATE_TIME" is unique
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(20)

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES
583293,R174,R034,00-00-03,125 ST,2019-06-08 00:00:00,2
194099,N045,R187,01-00-02,81 ST-MUSEUM,2019-06-01 17:00:00,2
193942,N045,R187,01-00-01,81 ST-MUSEUM,2019-06-02 17:00:00,2
194324,N045,R187,01-06-00,81 ST-MUSEUM,2019-06-11 21:00:00,2
660217,R249,R179,01-05-00,86 ST,2019-06-27 00:00:00,2
679477,R290,R161,00-00-00,KINGSBRIDGE RD,2019-06-07 05:00:00,2
660221,R249,R179,01-05-00,86 ST,2019-06-27 04:00:00,2
583125,R174,R034,00-00-02,125 ST,2019-06-08 00:00:00,2
193937,N045,R187,01-00-01,81 ST-MUSEUM,2019-06-01 17:00:00,2
194104,N045,R187,01-00-02,81 ST-MUSEUM,2019-06-02 17:00:00,2


In [19]:
# On 9/16, we seem to have two entries for same time.  Let's take a look
mask = ((turnstiles_df["C/A"] == "N045") & 
(turnstiles_df["UNIT"] == "R187") & 
(turnstiles_df["SCP"] == "01-00-00") & 
(turnstiles_df["STATION"] == "81 ST-MUSEUM") &
(turnstiles_df["DATE_TIME"].dt.date == datetime.datetime(2019, 6, 1).date()))
turnstiles_df[mask].head(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME
48238,N045,R187,01-00-00,81 ST-MUSEUM,BC,IND,06/01/2019,01:00:00,REGULAR,5253270,6696487,2019-06-01 01:00:00
48239,N045,R187,01-00-00,81 ST-MUSEUM,BC,IND,06/01/2019,05:00:00,REGULAR,5253277,6696494,2019-06-01 05:00:00
48240,N045,R187,01-00-00,81 ST-MUSEUM,BC,IND,06/01/2019,09:00:00,REGULAR,5253280,6696548,2019-06-01 09:00:00
48241,N045,R187,01-00-00,81 ST-MUSEUM,BC,IND,06/01/2019,13:00:00,REGULAR,5253356,6696877,2019-06-01 13:00:00
48242,N045,R187,01-00-00,81 ST-MUSEUM,BC,IND,06/01/2019,17:00:00,REGULAR,5253806,6697182,2019-06-01 17:00:00
48243,N045,R187,01-00-00,81 ST-MUSEUM,BC,IND,06/01/2019,17:00:00,RECOVR AUD,5253801,6697182,2019-06-01 17:00:00
48244,N045,R187,01-00-00,81 ST-MUSEUM,BC,IND,06/01/2019,21:00:00,REGULAR,5254017,6697270,2019-06-01 21:00:00


In [23]:
# Get rid of the duplicate entry
turnstiles_df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], 
                          inplace=True, ascending=False)
turnstiles_df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)

In [25]:
# Sanity Check to verify that "C/A", "UNIT", "SCP", "STATION", "DATE_TIME" is unique
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head()

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES
0,A002,R051,02-00-00,59 ST,2019-06-01 00:00:00,1
548916,R139,R031,04-00-01,34 ST-PENN STA,2019-06-28 12:00:00,1
548906,R139,R031,04-00-01,34 ST-PENN STA,2019-06-26 20:00:00,1
548907,R139,R031,04-00-01,34 ST-PENN STA,2019-06-27 00:00:00,1
548908,R139,R031,04-00-01,34 ST-PENN STA,2019-06-27 04:00:00,1


In [26]:
# Drop Exits and Desc Column.  To prevent errors in multiple run of cell, errors on drop is ignored
turnstiles_df = turnstiles_df.drop(["EXITS", "DESC"], axis=1, errors="ignore")

In [28]:
turnstiles_daily = (turnstiles_df
                        .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"],as_index=False)
                        .ENTRIES.first())

In [29]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES
0,A002,R051,02-00-00,59 ST,06/01/2019,7080817
1,A002,R051,02-00-00,59 ST,06/02/2019,7081567
2,A002,R051,02-00-00,59 ST,06/03/2019,7082949
3,A002,R051,02-00-00,59 ST,06/04/2019,7084565
4,A002,R051,02-00-00,59 ST,06/05/2019,7086109


In [30]:
turnstiles_daily[["PREV_DATE", "PREV_ENTRIES"]] = (turnstiles_daily
                                                       .groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE", "ENTRIES"]
                                                       .transform(lambda grp: grp.shift(1)))

In [31]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES
0,A002,R051,02-00-00,59 ST,06/01/2019,7080817,NaN,NaN
1,A002,R051,02-00-00,59 ST,06/02/2019,7081567,06/01/2019,7080817.0
2,A002,R051,02-00-00,59 ST,06/03/2019,7082949,06/02/2019,7081567.0
3,A002,R051,02-00-00,59 ST,06/04/2019,7084565,06/03/2019,7082949.0
4,A002,R051,02-00-00,59 ST,06/05/2019,7086109,06/04/2019,7084565.0


In [32]:
turnstiles_daily.tail()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES
136146,TRAM2,R469,00-05-01,RIT-ROOSEVELT,06/24/2019,5554,06/23/2019,5554.0
136147,TRAM2,R469,00-05-01,RIT-ROOSEVELT,06/25/2019,5554,06/24/2019,5554.0
136148,TRAM2,R469,00-05-01,RIT-ROOSEVELT,06/26/2019,5554,06/25/2019,5554.0
136149,TRAM2,R469,00-05-01,RIT-ROOSEVELT,06/27/2019,5554,06/26/2019,5554.0
136150,TRAM2,R469,00-05-01,RIT-ROOSEVELT,06/28/2019,5554,06/27/2019,5554.0


In [33]:
# Drop the rows for the earliest date in the df
turnstiles_daily.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

In [34]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES
1,A002,R051,02-00-00,59 ST,06/02/2019,7081567,06/01/2019,7080817.0
2,A002,R051,02-00-00,59 ST,06/03/2019,7082949,06/02/2019,7081567.0
3,A002,R051,02-00-00,59 ST,06/04/2019,7084565,06/03/2019,7082949.0
4,A002,R051,02-00-00,59 ST,06/05/2019,7086109,06/04/2019,7084565.0
5,A002,R051,02-00-00,59 ST,06/06/2019,7087657,06/05/2019,7086109.0


In [48]:
#create mask for the positive entries to delete neg entries from df. Roughly 1% of data.
positive_entries_mask = turnstiles_daily["ENTRIES"] > turnstiles_daily["PREV_ENTRIES"]
positive_entries_mask

1          True
2          True
3          True
4          True
5          True
6          True
7          True
8          True
9          True
10         True
11         True
12         True
13         True
14         True
15         True
16         True
17         True
18         True
19         True
20         True
21         True
22         True
23         True
24         True
25         True
26         True
27         True
29         True
30         True
31         True
          ...  
136120    False
136121    False
136122    False
136123    False
136125    False
136126    False
136127    False
136128    False
136129    False
136130    False
136131    False
136132    False
136133    False
136134    False
136135    False
136136    False
136137    False
136138    False
136139    False
136140    False
136141    False
136142    False
136143    False
136144    False
136145    False
136146    False
136147    False
136148    False
136149    False
136150    False
Length: 131260, dtype: b

In [55]:
#applied mask, this should only have positive counters. Daily data is CLEAN!
turnstiles_daily = turnstiles_daily[positive_entries_mask]
turnstiles_daily

/Users/alexlou/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES
1,A002,R051,02-00-00,59 ST,06/02/2019,7081567,06/01/2019,7080817.0
2,A002,R051,02-00-00,59 ST,06/03/2019,7082949,06/02/2019,7081567.0
3,A002,R051,02-00-00,59 ST,06/04/2019,7084565,06/03/2019,7082949.0
4,A002,R051,02-00-00,59 ST,06/05/2019,7086109,06/04/2019,7084565.0
5,A002,R051,02-00-00,59 ST,06/06/2019,7087657,06/05/2019,7086109.0
6,A002,R051,02-00-00,59 ST,06/07/2019,7089268,06/06/2019,7087657.0
7,A002,R051,02-00-00,59 ST,06/08/2019,7090115,06/07/2019,7089268.0
8,A002,R051,02-00-00,59 ST,06/09/2019,7090874,06/08/2019,7090115.0
9,A002,R051,02-00-00,59 ST,06/10/2019,7092269,06/09/2019,7090874.0
10,A002,R051,02-00-00,59 ST,06/11/2019,7093803,06/10/2019,7092269.0
